В этой тетрадке приведены примеры выполнения различных запросов к БД. Сами запросы можно посмотреть в файле _commands.py_, откуда они импортируются.

Начнем как раз с импорта всего необходимого: нам нужны модуль _mysql.connector_ для взаимодействия с базой данных, пароль пользователя (чтобы не а) хардкодить б) светить им), и сам список команд.

In [1]:
import mysql.connector
from conf import user_pswd
from commands import *

Создадим подключение и курсор для работы.

In [2]:
cnx = mysql.connector.connect(user='user', password = user_pswd, database='final_project')
cursor = cnx.cursor()

# Простые селекты

1. Получение данных о студенте
    * по id
    * по имени студента

In [3]:
stud_name = 'Петухова Маруся Мефодьевна'

In [4]:
def get_by_id(student_id):
    cursor.execute(student_by_id, (student_id,))
    
    for (name, group_num, year) in cursor:
        print("{}, группа {}, {}-й курс.".format(name, group_num, year))
    return

In [5]:
get_by_id(13)

Маликова Даша Оскаровна, группа 11, 1-й курс.


In [6]:
def get_by_name(name):
    cursor.execute(student_by_name, (name,))
    for (group_num, year) in cursor:
        print("{}, группа {}, {}-й курс.".format(name, group_num, year))
    return

In [7]:
get_by_name(stud_name)

Петухова Маруся Мефодьевна, группа 31, 3-й курс.


2. Получения списка
    * группы
    * курса

In [8]:
def get_group_list(group):
    cursor.execute(group_list, (group,))

    list_group = []
    for name in cursor:
        list_group.append(name[0])
    return list_group

In [9]:
list_group = get_group_list(42)
print("Студентов в группе: {}".format(len(list_group)))

Студентов в группе: 54


In [10]:
def get_year_list(year):
    cursor.execute(year_list, (year,))
    
    edu_year = []
    for name in cursor:
        edu_year.append(name[0])
    return edu_year

In [11]:
edu_year = get_year_list(3)
print('Студентов на курсе: {}'.format(len(edu_year)))

Студентов на курсе: 153


3. Простые селекты из нескольких таблиц
    * по имени преподавателя найти, какие он ведет предметы (2 таблицы)
    * вывести все общепотоковые предметы для какого-нибудь курса (2 таблицы)
    * вывести список студентов, посещающих какой-нибудь курс по выбору (3 таблицы)

In [12]:
prof_name = 'Павлюк Газинур Саддамович'

def get_courses_by_prof(name):
    cursor.execute(professors_courses, (prof_name,))
    
    list_courses = []
    for name in cursor:
        list_courses.append(name[0])
    return list_courses

In [13]:
prof_courses = get_courses_by_prof(prof_name)
for course in prof_courses:
    print(course)

Теоретические Теории Грамматики
Лексические Подходы Лингвистики


In [14]:
def courses_for_year(year):
    cursor.execute(year_courses, (year,))
    
    courses_year = []
    for name in cursor:
        courses_year.append(name[0])
    return courses_year

In [15]:
courses_year = courses_for_year(4)
for course in courses_year:
    print(course)

Динамические Теории Функций
Лексические Уравнения Вычислений
Статистические Уравнения Семантики
Функциональные Процессы Функций


In [16]:
choice_course = 'Аналитические Подходы Вычислений'

def list_by_choice(name):
    cursor.execute(choice_list, (name,))
    
    choice_students = []
    for name in cursor:
        choice_students.append(name)
    return choice_students

In [17]:
choice_students = list_by_choice(choice_course)

for (name, group_num, year) in choice_students:
    print("{}, группа {}, {}-й курс.".format(name, group_num, year))

Левина Милена Протасовна, группа 32, 3-й курс.
Петров Павел Алоизович, группа 23, 2-й курс.
Сулейманов Ильгам Назарович, группа 42, 4-й курс.
Исаков Адлан Павлинович, группа 12, 1-й курс.
Кудрявцева Наиля Гаврииловна, группа 12, 1-й курс.
Новикова Изабелла Адриановна, группа 23, 2-й курс.
Лебедев Федор Богданович, группа 41, 4-й курс.
Кузнецов Кирилл Альвианович, группа 42, 4-й курс.
Лукьянова Аниля Кадыровна, группа 11, 1-й курс.
Шульга Костя Львовна, группа 31, 3-й курс.


# Вложенный селект

По имени студента вывести все курсы, на которые он записан

In [18]:
def courses_for_student(name):
    cursor.execute(students_courses, (name,)*3)
    
    courses_list = []
    for name in cursor:
        courses_list.append(name)
    return courses_list

In [19]:
courses_list = courses_for_student(stud_name)
for (name, course_id) in courses_list:
    print("Код курса: {}\t Название курса: {}".format(course_id, name))

Код курса: 5	 Название курса: Аналитические Истоки Семантики
Код курса: 17	 Название курса: Аналитические Теории Синтаксиса
Код курса: 21	 Название курса: Функциональные Приложения Синтаксиса
Код курса: 29	 Название курса: Теоретические Приложения Лингвистики
Код курса: 43	 Название курса: Динамические Приложения Лингвистики
Код курса: 44	 Название курса: Динамические Направления Грамматики
Код курса: 45	 Название курса: Лексические Подходы Синтаксиса


# Удаление

"Уволим" преподавателей, которые ничего не ведут.

In [20]:
# для начала получим список преподавателей
def show_prof():
    show = "SELECT name FROM professors"
    cursor.execute(show)
    
    prof_list = []
    for name in cursor:
        prof_list.append(name[0])
    return prof_list

print("Всего преподавателей: {}".format(len(show_prof())))

Всего преподавателей: 30


In [21]:
fire_prof = ("DELETE FROM professors WHERE prof_id NOT IN"
               "(SELECT prof_id FROM courses)")
cursor.execute(fire_prof)

print("Всего преподавателей: {}".format(len(show_prof())))

Всего преподавателей: 25


# Вставка данных

Добавим нового студента.

In [22]:
def new_student(name, group_num, year):
    cursor.execute(add_student, (name, group_num, year))
    return print("Student added succesfully.")

In [23]:
new_stud = 'Григорьев Алексей Валерьевич'
new_student(new_stud, 32, 3)
get_by_name(new_stud)

Student added succesfully.
Григорьев Алексей Валерьевич, группа 32, 3-й курс.


# Изменение данных

Переведем только что добавленного студента в другую группу:

In [24]:
def change_group(group, name):
    cursor.execute(change_students_group, (group, name))
    return print("Student's group changed.")

In [25]:
change_group(31, new_stud)
get_by_name(new_stud)

Student's group changed.
Григорьев Алексей Валерьевич, группа 31, 3-й курс.


И на другой курс:

In [26]:
def change_year(year, name):
    cursor.execute("SELECT group_num, year FROM students WHERE name=%s", (name,))
    for result in cursor:
        old_group = result[0] % 10
    new_group = old_group + year*10
    cursor.execute(change_students_year, (new_group, year, name))
    return print('Changed successfully.')

In [27]:
change_year(4, new_stud)
get_by_name(new_stud)

Changed successfully.
Григорьев Алексей Валерьевич, группа 41, 4-й курс.


In [28]:
# так как это демонстрационная тетрадка, я не коммичу изменения в ДБ, а просто закрываю соединение
cursor.close()
cnx.close()